In [ ]:
# Kaggle / Colab setup script
# This cell clones the repository into the notebook environment, moves files
# (adjust paths as needed), upgrades pip and installs Python dependencies.
# NOTE: These are shell commands (bang `!`) intended for notebook execution.

# clone the ADIS repository (adjust URL if using a fork)
!git clone https://github.com/sathishkumar67/ADIS.git

# move the repository files into the current working directory (adjust if needed)
!mv /kaggle/working/ADIS/* /kaggle/working/ # change this path to move files to the current directory

# upgrade pip to latest
!pip install --upgrade pip

# install the required packages from requirements.txt
!pip install -r requirements.txt

In [ ]:
# Necessary imports for hyperparameter tuning and dataset handling
# - `joblib` is used to load saved Optuna studies (if any)
# - `optuna` provides hyperparameter optimization utilities
# - `hf_hub_download` downloads datasets/artifacts from the Hugging Face Hub
# - `unzip_file` (from utils) extracts the dataset archive with a progress bar
# - `YOLO11Model` is the project model wrapper used for training
from __future__ import annotations
import os
import joblib
import optuna
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model

In [ ]:
# Define global variables and dataset paths used by the notebook.
# Edit these values if your local layout differs.
REPO_ID = "pt-sk/ADIS"  # Hugging Face repo id for dataset (if used)
DATASET_NAME = "balanced_dataset"  # dataset archive name (without .zip)
FILENAME_IN_REPO = f"{DATASET_NAME}.zip"
LOCAL_DIR = os.getcwd()  # working directory inside the notebook
TRAIN_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/train"
VAL_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/val"
TEST_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 10
# class names corresponding to dataset labels (order matters)
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake']
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"

# Specify which YOLOv11 model weights to use
# Options: 'yolo11m.pt', 'yolo11s.pt', 'yolo11n.pt'
MODEL_PATH = ""

In [ ]:
# Download the dataset from the Hugging Face Hub and unzip it locally.
# This cell uses `hf_hub_download` to fetch the dataset archive and `unzip_file`
# (utility in this repository) to extract it with a progress bar.

hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)

# Extract the downloaded archive into the working directory
unzip_file(DATASET_PATH, LOCAL_DIR)

# Optionally remove the archive to free space
os.remove(f"{LOCAL_DIR}/{DATASET_NAME}.zip")
print("deleted dataset.zip")

# Print number of CPU cores available — used to set num_workers for dataloaders/training
num_cores = os.cpu_count()
print(f"No of Cores Available: {num_cores}")

In [ ]:
# Build and write a simple data YAML file that the YOLO training code expects.
# The YAML contains paths for train/val/test splits, number of classes (nc) and class names.
# Note: the triple-quoted f-string below will expand the variables defined earlier.

data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# Persist the data.yaml to disk so model code can read it
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)
    print("data yaml file written!.............")

In [ ]:
# If hyperparameter tuning has been performed previously, load the Optuna study to retrieve best params.
# NOTE: `study_path` should point to a joblib-serialized study file (.pkl).
study_path = ""
study = joblib.load(study_path)
best_params = study.best_params

# If you haven't run tuning, either leave study_path empty or provide a valid path.
# When `study_path` is empty this code will raise — it's included as an example of how to load results.

In [ ]:
# Configuration dictionary for training. Populate values from tuning or manual choices.
# Keys are typical training hyperparameters accepted by the YOLO11Model.train API in this repo.
CONFIG_DICT = {
"task": "detect",
"mode": "train",
"bohb": False,
"custom_callbacks": None,
"data": DATA_YAML_FILE,
"batch": 80, # batch size depending on GPU memory
"imgsz": 320, # image size depending on your constraints
"save": True,
"device": 0,
"workers": num_cores,
"pretrained": True,
"optimizer": "AdamW",
"seed": 42,
"epochs": 60, # epochs
"warmup_epochs": 5, # warmup
"patience": 5,
# If you have run tuning, these entries pull tuned values from the study.
"lr0":best_params["lr0"], # tuned learning rate, if not tuned, use default value or manual value
"lrf":best_params["lrf"], # tuned final learning rate, if not tuned, use default value or manual value
"weight_decay":best_params["weight_decay"], # tuned weight decay, if not tuned, use default value or manual value
"warmup_momentum":best_params["warmup_momentum"], # tuned warmup momentum, if not tuned, use default value or manual value
"momentum":best_params["momentum"]} # tuned momentum, if not tuned, use default value or manual value

In [ ]:
# Train the YOLOv11 model using the configuration above.
# `MODEL_PATH` can point to pretrained weights or be empty to rely on `pretrained=True`.
model = YOLO11Model(MODEL_PATH)
# The model's `train` method consumes the CONFIG_DICT keys; ensure they match the model API.
model.train(**CONFIG_DICT)

In [ ]:
# After training completes, the best performing model weights are typically saved to:
# `runs/detect/train/weights/best.pt` — check that path in your training output for the exact location.
# You can load that file for inference or to resume training later.